In [2]:
from __future__ import print_function
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN
from keras.models import Sequential
#from keras.utils.visualize_util import plot
import numpy as np

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
fin = open("Zhadan.txt", 'rb')
lines = []
for line in fin:
    line= line.strip().lower()
    line = line.decode("ascii","ignore")
    if len(line) == 0:
        continue
    lines.append(line)
fin.close()
text = " ".join(lines)

chars = set([c for c in text])
nb_chars = len(chars)
char2index = dict((c,i) for i,c in enumerate(chars))
index2char = dict((i,c) for i, c in enumerate(chars))

SEQLEN = 10
STEP = 1
input_chars = []
label_chars = []
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i:i+SEQLEN])
    label_chars.append(text[i+SEQLEN])
X= np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y=np.zeros((len(input_chars),nb_chars),dtype = np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1


In [4]:
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCH_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

model = Sequential()
model.add(SimpleRNN(HIDDEN_SIZE, return_sequences = False,
                   input_shape = (SEQLEN,nb_chars),
                   unroll = True))
model.add(Dense(nb_chars))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer = 'rmsprop')


In [5]:
for iteration in range(NUM_ITERATIONS):
    print("="*50)
    print("Iteration #: %d" % (iteration))
    model.fit(X,y,batch_size=BATCH_SIZE, epochs = NUM_EPOCH_PER_ITERATION)
    
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    print("Generating from seed: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):
        Xtest = np.zeros((1,SEQLEN,nb_chars))
        for i , ch in enumerate(test_chars):
            Xtest[0,i, char2index[ch]] = 1
        pred = model.predict(Xtest,verbose=0)[0]
        ypred = index2char[np.argmax(pred)]
        print(ypred, end="")
        # move forward with test_chars + ypred
        test_chars = test_chars[1:] + ypred
print()

Iteration #: 0
Epoch 1/1
27916/27916 [==============================] - 5s 171us/step - loss: 0.7571
Generating from seed:   , ,   , 
  , ,   ,                                                                                                     ==================================================
Iteration #: 1
Epoch 1/1
27916/27916 [==============================] - 4s 155us/step - loss: 0.6620
Generating from seed:  ,     ,  
 ,     ,     ,     ,     ,     ,     ,     ,     ,     ,     ,     ,     ,     ,     ,     ,     ,     ,     ,==================================================
Iteration #: 2
Epoch 1/1
27916/27916 [==============================] - 4s 160us/step - loss: 0.6338
Generating from seed:      . ,  
     . ,                                                                                                      ==================================================
Iteration #: 3
Epoch 1/1
27916/27916 [==============================] - 4s 159us/step - loss: 0.6196
Generating fro

KeyboardInterrupt: 